## A Hands-on Workshop series in Machine Learning
#### Instructor: Aashita Kesarwani

We have studied some binary classification algorithms in the previous session, for which the output variable $y$ is one of the classes - 0 or 1 for the input data $X$ and the function $f$ gives the decision boundary. 

### Classification with more than two classes

Suppose you want to use the texts written by 3 different authors to build a neural network that can predict which of those three authors have written an unclassified text.

This is a classification problem with three classes. The neural networks we have seen so far only has a single node in the output layer that gives the probability for the positive class for binary (two-class) classification.

For a multi-class (more than two classes) classification problem, the number of nodes in the output layer will be equal to the number of classes.

<img align="left" src="https://drive.google.com/uc?id=1J1JdmJqBOHSqBmEoj1WvYec2vatmIILa"  >

Each node will correspond to one of the classes and it will output the probability for that class.

For multiclass classification, we use softmax function instead of sigmoid for output layer and cross-entropy loss instead of log-loss for cost function. 

### Cross-entropy loss

For the binary classification, our label $y$ was simply taking binary integer values $0$ or $1$. For more than two classes, we use one-hot encoding. 

**One-hot encoding:**   
For $k$ classes, we use the k-dimensional vectors to represent each class where only one entry is 1 and others zero. For illustrion, let $k=3$

$$ y_1 = (1, 0, 0), \quad \quad \quad y_2 = (0, 1, 0), \quad \quad \quad y_3 = (0, 0, 1)$$


The **cross-entropy loss** for the $i^{th}$ sample is given by

$$ J^{(i)} = - \sum_{j=1}^k y^{(i)}_j \log(p^{(i)}_j)$$

The **average cross-entropy loss** across all samples is given by

$$ J = - \frac{1}{m} \sum_{i=1}^m \sum_{j=1}^k y^{(i)}_j \log(p^{(i)}_j)$$

where $y^{(i)}$ is the one-hot encoded label for the $i^{th}$ sample and $p^{(i)}_j$ is the probability of the $j^{th}$ class for the $i^{th}$ sample.  Note that the inner summation will only contain a single non-zero term.


Check that cross-entropy loss reduces to log loss for two classes.

### Softmax function
For a multi-class (more than two classes) problem, the number of nodes in the output layer are equal to the number of classes and we want the probabilities for each class to add up to $1$. One of the simple way to ensure this would be to divide the output for each node by the total sum of the outputs of all the nodes. This is called standard normalization.

$$ prob_i = \frac{z_i}{z_1 + \dots + z_n} $$

The preferred method for multi-class  classification problems is to use softmax function in the output layer. It converts the outputs, say $z_i$'s, into probabilities adding to $1$, by performing standard normalization on the exponentials of the outputs.

For each node, the softmax formula is

$$softmax(z_i) = \frac{e^{z_i}}{e^{z_1} + \dots + e^{z_n}} $$

$$ softmax([z_1, \dots, z_k]) = \left[\frac{e^{z_1}}{e^{z_1}+ \dots + e^{z_k}}, \dots, \frac{e^{z_k}}{e^{z_1}+ \dots + e^{z_k}}\right]$$

The exponentials in the softmax cancels the log in the cross-entropy loss/cost function defined above, causing loss to be linear in $z_i$ and thus, speeding up the backpropagation step.

The only assumption for softmax is that the examples cannot belong to two classes at the same time.

What do you get for softmax when there are only two classes? Convince yourself that softmax is an extension of sigmoid function for multi-class.

### Vanishing or Exploding Gradients

Deep Learning architectures, that is neural networks with many hidden layers, are driving the recent advances in the machine learning. 

<img align="center" src="https://drive.google.com/uc?id=1kcWsASHFLoEgRFNpi_cxgYUElzUvOYro" width=600 />
 
For deep neural networks, it can happen that the gradients are so small that the training process does not yield much positive results even after several epochs. The reason behind vanishing gradients lies in the backpropagation step. 

For a particular weight, say $w$, the weight update involves the partial derivate of the cost function, 

$$w := w - \alpha \frac{\partial J}{\partial w}$$

Computing the gradient $\frac{\partial J}{\partial W_n}$ for the weights in the very last layer $W_n$, 

$$ \frac{\partial J}{\partial W_n} = \frac{\partial J}{\partial y_{pred}} \frac{\partial y_{pred}}{\partial W_n}$$

Given the gradients for a layer, how do we compute the gradients for the preceding layer?  
Answer: Chain rule for derivatives, again.

$$ \frac{\partial J}{\partial W_{n-1}} = \frac{\partial J}{\partial W_n} \frac{\partial W_n}{\partial W_{n-1}}$$

When we look deeper into the calculations for the gradients, we see that it involves a series of derivatives multiplied to one another because of using the chain rule. 
Since each of these derivatives are influenced by the derivaties in the layer ahead of it, there are two possibilities:
* Gradients vanish to zero
$$ 0.9 * 0.8 * 0.7 * 0.6 * 0.5 * 0.4 * 0.3 * 0.2 * 0.1 \approx 0.0004$$
* Gradients blowing up to infinity
$$ 1 * 2 * 3 * 4 * 5 * 6 * 7 * 8 * 9 \approx 362880$$

The earlier layers in the network are far more affected by either of these problems than the layers closer to the output because the gradients are propagated backwards adding on the multiplying factors for the earlier layers of the network.

The vanishing gradients brings the process of updating weights to a standstill. Thus, running more epochs with training examples does not lead to the reduction in the cost function.


How do we accelerate the learning process of our network?
* Does the number of layers make a difference? 
* Does the number of nodes in each layer make a difference?
* Does weight initialization make a difference?
* Does the activation function make a difference?
* Does the learning rate make a difference?
* Does the scale of the features(variables) make a difference?

Let us revisit the sigmoid function. What do you observe about the derivative (slope) of the sigmoid function at different points?

$$sig(t) = \frac{1}{1+e^{-t}}$$

<img src="https://upload.wikimedia.org/wikipedia/commons/5/53/Sigmoid-function-2.svg" width=400 />

The derivative of the sigmoid function becomes vanishingly small in the saturating regions on both sides. This will prevent the weights from updating their values.

<img src="https://miro.medium.com/max/3946/1*6A3A_rt4YmumHusvTvVTxw.png" width=500 />

Note that the derivative of sigmoid function is $g'(x) = g(x)\left(1-g(x)\right)$ and its maximum value is $0.25$ when $x=0$. This reduces the values of the gradients in the backpropagation steps and thus adds to the vanishing gradient problem.

Hence, another activation function ReLU is preferred over the sigmoid activation. 

### Activation functions

#### Rectified Linear Units (ReLU):
Rectified Linear Units (ReLU) function is defined as:

\begin{equation}
g(x) = 
\begin{cases} 
x \text{ if } x \geq 0 \\
0 \text{ if } x < 0
\end{cases}
\end{equation}

<img src="https://miro.medium.com/max/2565/1*DfMRHwxY1gyyDmrIAd-gjQ.png" width=400 />

It is simply the identity function for positive values and hence, lets the weighted sum pass through it without modification but shuts down the activation when the weighted sum is negative.

It's derivative 
\begin{equation}
g'(x) = 
\begin{cases} 
1 \text{ if } x \geq 0 \\
0 \text{ if } x < 0
\end{cases}
\end{equation}

Other than addressing the vanishing gradient problem, ReLU also speeds up the training process significantly as compared to the sigmoid function and hence, it is the most commonly used activation function for the hidden layers.

**Cons:** 
* Unlike the sigmoid/softmax activation functions, ReLU is not suitable for the output layer of a network designed for classification. 
* ReLU can lead to dead neurons explained below.
* ReLU is not differentiable at $0$.

#### Dead neurons

ReLU as well as its derivative is zero for negative values. It means that for negative weighted sum, the output of the neuron will be zero and it will also not contribute towards updating weights connecting to it in the backpropagation step (check the derivation to see where the weighted sum comes into picture). Such a neuron is irreplacably dead. This is likely to happen if the biases are initialized to large negative values. Another likely scenario for dying neurons would be if the learning rate is too high. In this case, for a positive gradient and a small positive weight, a large update can result in negative weight which can in turn lead to negative weight sum and hence, dead neuron. 

$$ w := w - \alpha \frac{\partial J}{\partial w} $$

*He weight initialization* explained below is shown to address the problem of dead neurons. Lowering the learning rate $\alpha$ is helpful too. A variant of ReLU called Leaky ReLU can be used instead.

#### Leaky ReLU

Leaky ReLU tries to fix the dying ReLU problem by allowing a small constant slope $\alpha$ for negative values.

\begin{equation}
g(x) = 
\begin{cases} 
x \quad \text{ if } x \geq 0 \\
\alpha x \quad \text{if } x < 0
\end{cases}
\end{equation}

where $\alpha$ is a positive constant usually $0.01$.

<img src="https://ml-cheatsheet.readthedocs.io/en/latest/_images/leakyrelu.png" width=300 />

It's derivative
\begin{equation}
g'(x) = 
\begin{cases} 
1 \quad \text{if } x \geq 0 \\
\alpha \quad \text{ if } x < 0
\end{cases}
\end{equation}

<img src="https://ml-cheatsheet.readthedocs.io/en/latest/_images/leakyrelu_prime.png" width=300 />


#### Exponential Linear Unit (ELU):

Exponential Linear Unit (ELU) matches ReLU for positive values as they both are identity functions. For negative values, 

\begin{equation}
g(x) = 
\begin{cases} 
x \quad \quad \quad \quad \text{if } x \geq 0 \\
\alpha (1- e^x) \quad \text{ if } x < 0
\end{cases}
\end{equation}

where $\alpha$ is a positive constant, usually $1$.

<img src="https://ml-cheatsheet.readthedocs.io/en/latest/_images/elu.png" width=300 />

It's derivative
\begin{equation}
g'(x) = 
\begin{cases} 
1 \quad \quad \text{if } x \geq 0 \\
\alpha e^x \quad \text{ if } x < 0
\end{cases}
\end{equation}

<img src="https://ml-cheatsheet.readthedocs.io/en/latest/_images/elu_prime.png" width=300 />

ELU does not suffer from the problem of vanishing gradients or dead neurons (since the gradient is non-zero for negative values). Though it is computationally slower than ReLU, it convergences faster so overall computationally fast. It also often results in higher accuracy. For $\alpha=1$, it is continuous and differentiable at all points. It is very well suitable for deep neural networks.

### Weight initialization

To address the vanishing gradient problems, the general rule for weight initialization is:
* Generate a random sample from the standard normal distribution (also known as [Guassian distribution](https://www.khanacademy.org/math/statistics-probability/modeling-distributions-of-data/more-on-normal-distributions/v/introduction-to-the-normal-distribution))
* Multiply each weight by $\sqrt{\frac{2}{n_i}}$ where $n_i$ is the number of nodes in that layer 

More information on this topic can be found in [this paper](http://proceedings.mlr.press/v9/glorot10a.html) by Xavier Glorot and Yoshua Bengio, who introduced this rule. [This video](https://www.coursera.org/lecture/deep-neural-network/weight-initialization-for-deep-networks-RwqYe) on coursera is also helpful.

In Keras, this weight initialization rule can implemented by passing `kernel_initializer='glorot_normal'` to the `add(Dense())` function while defining the layers.

#### How to address overfitting?
- Reduce the number of features 
    - Discard some features
    - Dimensionality reduction techniques such PCA, LDA, etc.
- Simplify the model (by tuning hyperparameters)
- Reducing the number of epochs for training the network
- Regularization, Dropout, etc.
- Adding more training examples, if possible  
<img src="https://i.stack.imgur.com/rpqa6.jpg" width="450" height="600" />

In a nutshell, 
* **To reduce overfitting, reduce complexity.**
* **To reduce underfitting, increase complexity.**

### Addressing overfitting
#### Regularization
When we have weights that are higher in magnitude, the model is more likely to overfit, so we want to penalize the weights. This is achieved by adding regularization term to the error term in the cost function. 

There are two common ways to add the regularization term (using [$L1$ and $L2$-norms](https://machinelearningmastery.com/vector-norms-machine-learning/)) 

Cost function without regularization:
$$ J = \frac{1}{2 n} \sum_{i=1}^n (y^{(i)} - y_{pred}^{(i)})^2 $$

Cost function with L2 regularization:
$$ J = \frac{1}{2 n} \sum_{i=1}^n (y^{(i)} - y_{pred}^{(i)})^2 + c \sum_{j=1}^m w_j^2$$

Cost function with L1 regularization:
$$ J = \frac{1}{2 n} \sum_{i=1}^n (y^{(i)} - y_{pred}^{(i)})^2 + c \sum_{j=1}^m |w_j|$$

As the training process tries to minimize the cost function, the regularization term ensures that the weights are kept small and thereby simplifies the model.

Forward propagation step remains unchanged, whereas the formulae for the backpropagation step that contains the gradients of the cost w.r.t weights changes:
$$ w := w - \alpha \frac{\partial J}{\partial w}$$

This technique of regularization using either $L1$ or $L2$-norm is one of the two most commonly used techniques to address overfitting in deep neural networks. The other one being Dropout regularization.


#### Dropout

Drop-out means dropping out (or ignoring) some randomly chosen nodes during the training of the neural network. 

* For each iteration in the training, some nodes are dropped (or their activations are switched off). 
* The same nodes are dropped for both forward and backward propagation. 
* The nodes to be dropped off are chosen at random. Each node is given the probability $p$ that it will be kept and the probability $1-p$ that it will be dropped. Whether each node would be dropped or not is chosen at random with the given probabilities. This results in roughly $1-p$ proportion of nodes being dropped out in each iteration.
* The dropout is implemented on all layers, except the output layer since it would not make much sense to drop the output nodes.
* The dropout is implemented only during the training phase. It is not used for the predictions once the weights are trained.


The network learns multiple independent representation and hence, it is less sensitive to specific weights, thus reducing overfitting. In some ways, a network with dropout is like having an ensemble of neural networks.

### Tuning the number of epochs

It is helpful to keep a check on both training and validation errors by printing them out at regular intervals and stop the training process at a suitable time to avoid both underfitting and overfitting to the training set.

<img src="http://fouryears.eu/wp-content/uploads/2017/12/early_stopping.png" width=400 />

#### Check out [Tensorflow Playground](https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=10&networkShape=4,2&seed=0.93390&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false)!!